In [0]:
from tensorflow.python.client import device_lib

In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16909773848383913035, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3404347468619794679
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5566882350761235352
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13871360953381696079
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

W0725 03:52:46.284954 139868896839552 deprecation.py:323] From <ipython-input-4-2aed5ca29197>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 03:52:46.287113 139868896839552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 03:52:46.292593 139868896839552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0725 03:52:46.826914 139868896839552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0725 03:52:47.008435 139868896839552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:

# to get train images

mnist.train._images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:

# to get train images

mnist.train._images.shape

(55000, 784)

In [9]:
import numpy as np

np.unique(mnist.train._labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [0]:
import tensorflow as tf
class FeedForward:
  def __init__(self, learning_rate):
    self.X = tf.placeholder(tf.float32, [None, 784])
    self.Y = tf.placeholder(tf.int32, [None])
    feed1 = tf.layers.dense(self.X, 256, 
                            activation = tf.nn.relu)
    feed2 = tf.layers.dense(feed1, 128,
                           activation = tf.nn.relu)
    self.logits = tf.layers.dense(feed2, 10)
    self.cost = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels = self.Y, logits = self.logits)
    self.cost = tf.reduce_mean(self.cost)
    self.optimizer = tf.train.AdamOptimizer(learning_rate) \
    .minimize(self.cost)
    correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
    self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
learning_rate = 1e-3
batch_size = 128
epoch = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = FeedForward(learning_rate)
sess.run(tf.global_variables_initializer())

W0725 04:39:48.547187 139868896839552 deprecation.py:323] From <ipython-input-12-10288bf35d13>:7: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0725 04:39:48.552919 139868896839552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model.logits #Pytorch can be used directly

<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 10) dtype=float32>

In [0]:
from tqdm import tqdm

In [17]:
for e in range(epoch):
  pbar = tqdm(
        range(0, len(mnist.train._labels), batch_size), desc = 'train loop'
    )
  train_loss, train_acc, test_loss, test_acc = [], [], [], []
  for i in pbar:
    index = min(i + batch_size, len(mnist.train._labels))
    batch_x = mnist.train._images[i: index]
    batch_y = mnist.train._labels[i: index]
    _, cost, accuracy = sess.run([model.optimizer, model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    train_loss.append(cost)
    train_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)

  pbar = tqdm(
        range(0, len(mnist.test._labels), batch_size), desc = 'test loop'
    )  
  for i in pbar:
    index = min(i + batch_size, len(mnist.test._labels))
    batch_x = mnist.test._images[i: index]
    batch_y = mnist.test._labels[i: index]
    cost, accuracy = sess.run([model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    test_loss.append(cost)
    test_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)
  
  
  print('epoch: %d, avg train loss: %f, avg train accuract: %f'%(e + 1,
                                                             np.mean(train_loss),
                                                             np.mean(train_acc)))
  print('epoch: %d, avg test loss: %f, avg test accuract: %f'%(e + 1,
                                                             np.mean(test_loss),
                                                             np.mean(test_acc)))

train loop:   5%|▍         | 20/430 [00:00<00:02, 195.41it/s, accuracy=0.953, cost=0.162]

epoch: 1, avg train loss: 0.318057, avg train accuract: 0.910422
epoch: 1, avg test loss: 0.159613, avg test accuract: 0.950455


train loop:   4%|▍         | 19/430 [00:00<00:02, 188.12it/s, accuracy=0.977, cost=0.0701]

epoch: 2, avg train loss: 0.126808, avg train accuract: 0.962683
epoch: 2, avg test loss: 0.118983, avg test accuract: 0.962322


train loop:   6%|▌         | 24/430 [00:00<00:01, 232.09it/s, accuracy=0.961, cost=0.0986]

...

epoch: 8, avg train loss: 0.019054, avg train accuract: 0.994123
epoch: 8, avg test loss: 0.090511, avg test accuract: 0.974782


train loop:   5%|▌         | 22/430 [00:00<00:01, 213.59it/s, accuracy=0.992, cost=0.0253]

epoch: 9, avg train loss: 0.017914, avg train accuract: 0.994287
epoch: 9, avg test loss: 0.085381, avg test accuract: 0.977156


test loop: 100%|██████████| 79/79 [00:00<00:00, 258.05it/s, accuracy=1, cost=0.0113]


epoch: 10, avg train loss: 0.014652, avg train accuract: 0.995094
epoch: 10, avg test loss: 0.087282, avg test accuract: 0.977947
